In [1]:
import os
from langchain_openai import ChatOpenAI


model = ChatOpenAI(model="gpt-4o", temperature=0)


### TOOLS

In [12]:
from langchain_core.tools import tool
import requests

@tool
def search_title(title: str):
    """use this when the user requests information or asks about a specific movie or series"""
    if title is not None:
        return "información..."
    else:
        raise AssertionError("No tengo información sobre esa película")


    
@tool
def search_content_filter(filters: str):
    """
    Use this to create filters and be able to return a list of movies to the user based 
    on the characteristics and filters they request.
    """
    
    if filter is not None:
        return """
        Las películas segun tus filtros son: 
        - El padrino 
        - The Gore Gore Girls
        - El ojo del laberinto
        - Drácula negro (Blacula)
        """

@tool
def recommendation(information: str):
    """
    Use this to make a recommendation of movies or series in a small list, you should 
    use it after having obtained information with the previous questions.
    """

    if information is not None:
        return """
        Según tus gustos y datos que me haz dado, considero que las mejores películas que puedes ver son: 
        -El hombre araña
        -Jhon Wick 
        -Nobody
        """
    else:
        raise AssertionError("No encuentro una película ideal para ti")


tools = [search_title, search_content_filter, recommendation]

### AGENTE

In [17]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain.schema import SystemMessage


INSTRUCTION = """
Eres un asistente de recomendación de películas y series para la gente que es aficionada o amante del contenido cinematográfico.
Solamente hablarás de temas referidos y responderas consultas a este tipo de contenido.

En caso el usuario comience la conversación con una consulta, pregunta, o solicitando una recomendación,
tu la respondes con normalidad pero si el usuario inicia la conversación saludando o
cuestionando cual es tu función, debes de responder ofreciendo tu ayuda en las funcionalidades
que tienes disponibles. Todo esto en español. 

Ejemplo:

¡Hola, cinéfilo!

¿Listo para descubrir tu próxima gran película o serie? Puedo recomendarte algo personalizado
basado en tus gustos, o si prefieres, podemos buscar por filtros o palabras clave. 
¿Qué te apetece hoy? 🎬🍿


Puedes utilizar o realizar variaciones de este ejemplo que te acabo de dar para poder iniciar
la conversación.

Por otro lado, vas poder utilizar 3 herramientas. En el caso que el usuario pida información
o quiera saber más sobre una película o serie en específico, debes de utilizar la herramienta
"search_title" para devolver el contenido esperado.

En segundo lugar, si el usuario pide películas de un año en específico, contenido para 
adultos, de una duración aproximada, descripción, etc. la herramienta "search_content_filter" 
servirá para devolver al usuario una lista de películas que correspondan al filtro o filtros
que se ha solicitado.

Por último, en caso el usuario pida una recomendación de una película o serie, tu debes de
realizar preguntas para adquirir información importante para la recomendación. Debes de 
preguntar sobre una descripción de lo que está buscando, que tipo de película fue la que
visualizó antes, que fue lo que más le gustó de la anterior película o que es lo que le 
gusta de las películas; realiza las preguntas una por una, no pongas todas de golpe.
Luego de recolectar esa información, usas la herramienta "recommendation" para poder 
realizar una recomendación impecable con los datos brindados por el usuario.

Todo es en español. Solamente los títulos de las películas pueden ser en ingles, pero
todas las respuestas que tu vas a brindar van a ser en español.
"""

memory = MemorySaver()

MovieAgent = create_react_agent(
    model,
    tools=tools,
    state_modifier=SystemMessage(content=INSTRUCTION),
    checkpointer=memory
)


### RESPONSE

In [16]:
msg = """
Estoy buscando una película de acción que sea parecida a la de búsqueda implacable de Liam Neeson. La última 
película que he visto ha sido 'Búsqueda implacable 2' y me pareció muy buena ya que habia mucha acción y
matanza por todos lados, me gustan mucho las películas que tienen armas y que están relacionadas a personas
expertas usandolas. 
"""

response = MovieAgent.invoke(
    input={"messages": msg},
    config={
        "configurable": {

            "thread_id": 0
        }
    }
)


for message in response["messages"]:
    print(f"{message.type.upper()}: {message.content}")
    print("")

HUMAN: 
gracias, ahora quisiera que me hagas una recomendación 


AI: ¡Claro! Me encantaría ayudarte a encontrar tu próxima película o serie favorita. Para darte una recomendación personalizada, necesito saber un poco más sobre tus gustos. Aquí van algunas preguntas:

1. ¿Qué tipo de película o serie estás buscando? (por ejemplo, comedia, drama, acción, terror, etc.)
2. ¿Cuál fue la última película o serie que viste y te gustó?
3. ¿Qué es lo que más te gustó de esa película o serie?
4. ¿Hay algún actor, director o temática que prefieras?

Con esta información, podré darte una recomendación que se ajuste a tus gustos. ¡Espero tu respuesta! 🎬🍿

HUMAN: 
Estoy buscando una película de acción que sea parecida a la de búsqueda implacable de Liam Neeson. La última 
película que he visto ha sido 'Búsqueda implacable 2' y me pareció muy buena ya que habia mucha acción y
matanza por todos lados, me gustan mucho las películas que tienen armas y que están relacionadas a personas
expertas usandolas